In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
plt.rcParams['figure.figsize'] = (30, 30)

import wikipedia

import re
import spacy
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import networkx as nx

from umap import UMAP
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.spatial.distance import cosine

from tqdm import tqdm_notebook as tqdm

In [ ]:
nlp = spacy.load('en')

In [ ]:
df = pd.read_json('../data/calm_records.json')

### get average word vector for our collection-level record

In [ ]:
record = df.loc[269229]['AdminHistory'][0]
soup = BeautifulSoup(record, 'html.parser')
plain_text = soup.get_text()
plain_text = ' '.join(plain_text.split())


print(plain_text)

In [ ]:
doc = nlp(plain_text)
doc_avg_wv = np.array([word.vector for word in doc]).mean(axis=0)

### get average word vector for a specific, known wikipedia page

In [ ]:
h = wikipedia.page('Hinge_loss')
doc = nlp(h.content)
np.array([word.vector for word in doc]).mean(axis=0).shape

In [ ]:
known_page = nlp(wikipedia.page('dog').content)
known_avg_wv = np.array([word.vector for word in known_page]).mean(axis=0)

### get average word vectors for all pages in wikipedia search 

In [ ]:
wikipedia.search('pool')

In [ ]:
possible_avg_wvs = pd.Series()
possible_page_titles = [title for title in wikipedia.search('chicken')
                        if 'disambiguation' not in title]
print(possible_page_titles)

In [ ]:
def get_pages(page_name):
    pages = []
    try:
        wikipedia.page(page_name).url
        # we're just hitting the url to check for disambiguation errors
        pages.append(page_name)
    except wikipedia.exceptions.DisambiguationError as disambiguation:
        for option in disambiguation.options:
            # Note that we're only going one level deep into disambiguations.
            # This should be more than enough for our purposes, and it's easy
            # to get caught in horrible, endless loops and branches if we make
            # this properly recursive.
            try: 
                wikipedia.page(option).url
                pages.append(option)
            except wikipedia.exceptions.DisambiguationError: pass
    return pages

In [ ]:
def flatten(list_of_lists):
    return [j for i in list_of_lists for j in i]

In [ ]:
all_page_names = np.unique(flatten([get_pages(page_name) 
                                    for page_name in possible_page_titles]))

In [ ]:
known_page = nlp(wikipedia.page('dog').summary)
known_avg_wv = np.array([word.vector for word in known_page]).mean(axis=0)

In [ ]:
candidate_wvs = pd.Series({page_name: (np.array([word.vector for word in 
                                                 nlp(wikipedia.page(page_name).content)])
                                       .mean(axis=0))
                           for page_name in tqdm(all_page_names)})

in our example we have a load of text about swimming, and we want to figure out which version of the page 'pool' is most relevant

In [ ]:
df = candidate_wvs.to_frame('word vector')

df['similarity'] = df['word vector'].apply(lambda avg_wv: cosine(avg_wv, known_avg_wv))
df.sort_values(by='similarity')

In [ ]:
wikipedia.page('Chicken wings as food').url

In [ ]:
for ent in nlp(plain_text).ents:
    print(ent)

In [ ]:
all_page_names = np.unique(flatten([get_pages(page_name) 
                                    for page_name in wikipedia.search(str(nlp(plain_text).ents[0]))]))

candidate_wvs = pd.Series({page_name: (np.array([word.vector for word in 
                                                 nlp(wikipedia.page(page_name).content)])
                                       .mean(axis=0))
                           for page_name in tqdm(all_page_names)})

In [ ]:
df = candidate_wvs.to_frame('word vector')

df['similarity'] = df['word vector'].apply(lambda avg_wv: cosine(avg_wv, doc_avg_wv))
df.sort_values(by='similarity')

In [ ]:
wikipedia.search('dog')

In [ ]:
string_2 = wikipedia.page('chicken').summary

# try (ratio'd) set intersection of words

# try tfidf

# try glove

In [ ]:
with open('/Users/pimh/Downloads/glove.6B/glove.6B.300d.txt') as f:
    glove = {}
    for line in tqdm(f.read().split('\n')):
        try:
            line = line.split()
            id = line[0]
            wv = np.array(line[1:]).astype(np.float32)
            glove[id] = wv
        except: pass

In [ ]:
wikipedia.page('swimming').summary

In [ ]:
candidate_page_summaries = pd.Series({page: wikipedia.page(page).content.lower()
                                      for page in tqdm(get_pages('amazon'))})

In [ ]:
known_summary = wikipedia.page('rainforest').content.lower()
known_wv = get_doc_vector(known_summary)

In [ ]:
def get_doc_vector(input_text):
    return np.stack([glove[str(word)] for word in nlp(input_text)
                     if str(word) in glove.keys()]).mean(axis=0)

In [ ]:
frame = candidate_page_summaries.apply(get_doc_vector).to_frame('wv')

In [ ]:
frame['similarity'] = frame['wv'].apply(lambda x: cosine(x, known_wv))

In [ ]:
frame.sort_values(by='similarity')